<a href="https://colab.research.google.com/github/JeetRoy97/Democracy-2.0/blob/main/langauage_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#!pip install git+https://github.com/allenai/longformer.git

# Language Detection

In [ ]:
!pip install pycld2

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Research/Other datasets/union_v2.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
from nltk.tokenize import sent_tokenize
s = list(df["title"].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack())

In [ ]:
s[:10]

['Under Income Tax, all medical related deductions to be clubbed with a much higher overall limit.',
 'Recommendations',
 'Rental deduction and deduction towards own housing for self-occupation should be clubbed and given a higher limit.',
 'House Rent Income and interest expense to be treated separately in the existing section.',
 'The deductions and exemptions related to Housing for Individuals are spread across many sections with varying limits.',
 'Salaried people with an allowance "named" House Rent Allowance get an exemption under Section 10 with regard to rent expenses paid.',
 'Others who do not get a House Rent Allowance get a deduction under Section 80GG for the same.',
 'Those who buy or build a house or repay housing loan principal get a deduction under section 80C within the overall limit of Rs.150000 bundled with many other investments / deductions, some of which actually cover the whole limit.',
 '(There is another section on affordable housing but that is specific to fi

In [ ]:
df_senttoken = pd.DataFrame()
df_senttoken['title'] = s

In [ ]:
# import pycld2 as cld2
# text_content = df['title'][8]
# _, _, _, detected_language = cld2.detect(text_content,  returnVectors=True)
# temp=[detected_language[i][2] for i in range(len(detected_language))]
# print(detected_language, temp)
# temp = np.array(temp)
# if 'Unknown' in np.unique(temp) and len(temp) > 1:
#   temp = temp[~np.isin(temp, 'Unknown')]

In [ ]:
def lang_detect(text_content):
  _, _, _, detected_language = cld2.detect(text_content,  returnVectors=True)
  #print(detected_language)
  temp = [detected_language[i][2] for i in range(len(detected_language))]
  temp = np.array(temp)
  if 'Unknown' in np.unique(temp) and len(temp) > 1:
    temp = temp[~np.isin(temp, 'Unknown')]
  temp = set(temp)
  res = ",".join(temp)
  #print(res)
  try:
    return res
  except:
    return 'ENGLISH'

In [ ]:
import pycld2 as cld2
new = df_senttoken['title'].apply(lang_detect)

In [ ]:
df_senttoken['lang'] = new

In [ ]:
df_senttoken['lang'].value_counts()

ENGLISH             2887
Unknown              384
                     178
HINDI                113
ENGLISH,HINDI         17
KANNADA                4
MARATHI                3
BENGALI                2
TAMIL                  1
ORIYA                  1
XHOSA                  1
X_Inherited            1
ENGLISH,GUJARATI       1
ENGLISH,NEPALI         1
Name: lang, dtype: int64

In [ ]:
df_senttoken.loc[df_senttoken['lang'] == 'XHOSA' ]

,title,lang
980,2Lakhs.,XHOSA


In [ ]:
df_senttoken.loc[df_senttoken['lang'] == 'X_Inherited' ]

,title,lang
2404,!👨‍🎓👩‍🎓👨‍⚕️👩‍⚕️👨🏻‍🔬👩🏼‍🔬👩🏻‍🚀,X_Inherited


In [ ]:
df_senttoken.loc[df_senttoken['lang'] == 'Unknown' ]

,title,lang
1,Recommendations,Unknown
13,Pls...,Unknown
20,in terms of capital investment.,Unknown
28,to come under exemption.,Unknown
31,Govt.,Unknown
...,...,...
3554,Happy new year Sir,Unknown
3557,good,Unknown
3564,Thank you,Unknown
3582,It was last revised in 2015.,Unknown


In [ ]:
df_senttoken.loc[df_senttoken['lang'] == '' ]

,title,lang
37,2.,
40,4.,
101,2.,
103,3.,
108,2.,
...,...,...
3566,1).,
3568,2).,
3570,3).,
3572,4).,


In [ ]:
# df = df.replace('GUJARATI,ENGLISH', 'ENGLISH')
# df = df.replace('NEPALI,ENGLISH', 'ENGLISH')
# df = df.replace('Unknown', 'ENGLISH')

In [ ]:
# df_senttoken = df_senttoken.loc[df_senttoken['lang'] != '' ]
# len(df)

858

In [ ]:
df_senttoken.to_csv("/content/drive/MyDrive/Research/Other datasets/union_v3.csv", index = False)

# Pre-processing

In [ ]:
# !pip install tensorflow-text

In [ ]:
# import tensorflow_hub as hub
# import tensorflow as tf
# import tensorflow_hub as hub
# import numpy as np
# import tensorflow_text
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
# model = hub.load(module_url)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Research/Other datasets/union_v3.csv")

In [ ]:
import re
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet') 
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    tweet = re.sub(r'\n', '', (phrase))
    tweet = re.sub(r'\t\w*', '', (tweet))
    tweet = re.sub(r'-', '', tweet)
    tweet = re.sub(r'  ', '', tweet)
    tweet = re.sub(r'\\n', ' ', tweet)
    tweet = re.sub(r'\n', ' ', tweet)
    tweet = re.sub(r',', '', tweet)
    #tweet = re.sub('\w+','', string)
    tweet = re.sub(r'\[', '', tweet)
    tweet = re.sub(r'\]', '', tweet)
    tweet = re.sub(r'\(', '', tweet)
    tweet = re.sub(r'\)', '', tweet)
    
    




    tweet = re.sub(r'_', '', tweet)
    tweet = re.sub(r'"', '', tweet)
    tweet = re.sub(r'&', '', tweet)
    tweet = re.sub(r'-', '', tweet)
    tweet = re.sub(r';', '', tweet)
    tweet = re.sub(r'``', '', tweet)
    tweet = re.sub('\'','', tweet)
    # general
    phrase = re.sub(r"n\'t", " not", tweet)
    phrase = re.sub(r"pm", " prime minister", tweet)
    phrase = re.sub(r"fm", " finance minister", tweet)
    phrase = re.sub(r"govt", " government", tweet)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess(doc_data):
  print("Preprocessing .....")
  from tqdm import tqdm
  preprocessed_reviews = []
  for sentance in tqdm(doc_data['title'].values):
      sentance = decontracted(sentance)
      sentance = re.sub("\S*\d\S*", "", sentance).strip()
      sentance = re.sub('[^A-Za-z]+', ' ', sentance)
      sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
      sentance = ' '.join(lemmatizer.lemmatize(word, pos ="v") for word in sentance.split())
      preprocessed_reviews.append(sentance.strip())
  return preprocessed_reviews

In [ ]:
df_en = df[:][df['lang'] == 'ENGLISH']

In [ ]:
len(df_en)

2887

In [ ]:
df_en = df_en.drop_duplicates()
len(df_en)

2795

In [ ]:
sents = preprocess(df_en)
print(len(sents))
#sent_vect = model(sents)

Preprocessing .....


100%|██████████| 2795/2795 [00:00<00:00, 6907.66it/s]

2795


In [ ]:
df_en['preprocessed'] = sents

In [ ]:
df_hi = df[:][df['lang'] == 'HINDI']

In [ ]:
df_en.to_csv('/content/drive/MyDrive/Research/Other datasets/df_en.csv', index = False)
df_hi.to_csv('/content/drive/MyDrive/Research/Other datasets/df_hi.csv', index = False)

In [ ]:
#sents

In [ ]:
# sent_vect = model(sents)
# print(sent_vect[0].shape)
# sent_vec_n = []
# for i in range(len(sent_vect)):
#   sent_vec_n.append(np.asarray(sent_vect[i]))
# df_en['sent_vec'] = sent_vec_n

# Clustering

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/Research/Other datasets/union.csv")

In [ ]:
# !pip install hdbscan

In [ ]:
# df_en['sent_vec'].head()

In [ ]:
# import hdbscan
# clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
# cluster_labels = clusterer.fit_predict(sent_vec_n)

In [ ]:
# cluster_labels.unique()

In [ ]:
# import hdbscan
# from sklearn.datasets import make_blobs

# data, _ = make_blobs(1000)

# clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
# cluster_labels = clusterer.fit_predict(data)

In [ ]:
# data

In [ ]:
# from sklearn.metrics import silhouette_score
# from sklearn.cluster import KMeans

# sil = []
# kmax = 10

# # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
# for k in range(2, kmax+1):
#   kmeans = KMeans(n_clusters = k).fit(sent_vec_n)
#   labels = kmeans.labels_
#   sil.append(silhouette_score(sent_vec_n, labels, metric = 'euclidean'))
# sil

In [ ]:
# result = np.where(sil == np.amax(sil))
# cl=int(result[0])
# #type(cl)

In [ ]:
# import matplotlib.pyplot as plt 
# x = range(2,len(sil)+2)
# y = sil 
# plt.plot(x, y) 
# plt.xlabel('No. of clusters') 
# plt.ylabel('Silhouette score')  
# plt.title('Silhouette graph')  
# plt.show() 

In [ ]:
!pip install bertopic

     |████████████████████████████████| 57 kB 6.4 MB/s 
     |████████████████████████████████| 5.2 MB 60.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 3.5 MB 55.6 MB/s 
     |████████████████████████████████| 1.2 MB 49.3 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 636 kB 59.0 MB/s 
     |████████████████████████████████| 6.8 MB 48.6 MB/s 
     |████████████████████████████████| 895 kB 63.0 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330763 sha256=15c11101f26ee8959157e9696abff3d7bdb1acf8dba18666e758eb7bdcec4854
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(sents)

# topic_model = BERTopic(language="distilbert-base-nli-mean-tokens", calculate_probabilities=True, verbose=True)
# topics, probs = topic_model.fit_transform(sents)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2022-02-28 08:49:52,170 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-02-28 08:50:04,214 - BERTopic - Reduced dimensionality with UMAP
2022-02-28 08:50:04,308 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
freq = topic_model.get_topic_info();
freq

,Topic,Count,Name
0,-1,242,-1_tax_government_income_increase
1,0,187,0_not_tax_government_people
2,1,120,1_india_budget_make_government
3,2,55,2_reduce_tax_consider_products
4,3,49,3_tax_income_limit_increase
5,4,27,4_waste_book_water_plastic
6,5,21,5_bsnl_happy_good_abiramam
7,6,18,6_increase_limit_section_tax
8,7,14,7_nadu_tamil_tobacco_finance
9,8,14,8_ko_ki_ke_se


In [ ]:
df_en['topic'] = topics  # Select the most frequent topic

In [ ]:
#topic_model.visualize_topics()

In [ ]:
# topic_model.visualize_distribution(probs[200], min_probability=0.015)

In [ ]:
# topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
# topic_model.visualize_barchart(top_n_topics=5)

In [ ]:
# topic_model.visualize_heatmap(n_clusters=3, width=1000, height=1000)

In [ ]:
# topic_model.visualize_term_rank()

In [ ]:
# similar_topics, similarity = topic_model.find_topics("medical", top_n=5); similar_topics

In [ ]:
topic_model.get_topic(-1)

[('tax', 0.04508775017330836),
 ('government', 0.024190694230277594),
 ('income', 0.022329077079109267),
 ('increase', 0.021416666687272882),
 ('bank', 0.019149118747686787),
 ('people', 0.018116572469211455),
 ('make', 0.017114562065573193),
 ('give', 0.01701353353879424),
 ('india', 0.01693973839441207),
 ('get', 0.0167323140193219)]

In [ ]:
# topic_model.update_topics(sents, topics, n_gram_range=(1, 2))

In [ ]:
# topic_model.get_topic(4)

In [ ]:
gk = df_en.groupby('topic')

In [ ]:
l=df_en['topic'].unique()

In [ ]:
df_test=gk.get_group(l[-1])
df_test = df_test.reset_index(drop=True)

In [ ]:
df_test.to_csv("/content/drive/MyDrive/Research/Other datasets/test_topic.csv")

In [ ]:
df_test

,title,lang,preprocessed,topic
0,Taxpayer ki problem ko km krne wala budget hon...,ENGLISH,taxpayer ki problem ko km krne wala budget hon...,8
1,Emergency credit seva (10000) lagu hona chahiy...,ENGLISH,emergency credit seva lagu hona chahiye bina b...,8
2,jaise bmc ko safai ke liye anudaan diya waise ...,ENGLISH,jaise bmc ko safai ke liye anudaan diya waise ...,8
3,mananiya madam aaj ki mehgai ke hisab se incom...,ENGLISH,mananiya madam aaj ki mehgai ke hisab se incom...,8
4,xjfjcjsjkzkxvkdmmxmvgmkvkvkckdkfvkvkckfngnjvvj...,ENGLISH,xjfjcjsjkzkxvkdmmxmvgmkvkvkckdkfvkvkckfngnjvvj...,8
5,kissano ka mitr bujat pase kare 🙏,ENGLISH,kissano ka mitr bujat pase kare,8
6,bhartiya sanbidhan ke anuched 112 me Bharat ke...,ENGLISH,bhartiya sanbidhan ke anuched bharat ke kendri...,8
7,"Sir,\nBudget me education pr tax kam hona chah...",ENGLISH,sirbudget education pr tax kam hona chahiye ed...,8
8,1 bharat sarkar ko employments k avsar badne c...,ENGLISH,bharat sarkar ko employments k avsar badne gov...,8
9,My respected sir\nI am farmar man sir kisano k...,ENGLISH,respect siri farmar man sir kisano ki aye ko g...,8


In [ ]:
#!pip install -r requirements.txt